In this notebook we will make a custom environment on similar structure of **open-ai-gym** environment and use **Stable Baselnes 3(SB3)** to learn.

In [1]:
import gym
from gym import spaces
import numpy as np
import cv2
import random
import time
from collections import deque
from stable_baselines3 import PPO
import os

In [2]:
SNAKE_LEN_GOAL = 30

In [3]:
def collision_with_apple(apple_position, score):
    apple_position = [random.randrange(1,50)*10,random.randrange(1,50)*10]
    score += 1
    return apple_position, score

def collision_with_boundaries(snake_head):
    if snake_head[0]>=500 or snake_head[0]<0 or snake_head[1]>=500 or snake_head[1]<0 :
        return 1
    else:
        return 0

def collision_with_self(snake_position):
    snake_head = snake_position[0]
    if snake_head in snake_position[1:]:
        return 1
    else:
        return 0

In [4]:
class SnekEnv(gym.Env):

    def __init__(self):
        super(SnekEnv, self).__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        self.action_space = spaces.Discrete(4)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=-500, high=500,
                                            shape=(5+SNAKE_LEN_GOAL,), dtype=np.float32)

    def step(self, action):
        self.prev_actions.append(action)
        cv2.imshow('a',self.img)
        cv2.waitKey(1)
        self.img = np.zeros((500,500,3),dtype='uint8')
        # Display Apple
        cv2.rectangle(self.img,(self.apple_position[0],self.apple_position[1]),(self.apple_position[0]+10,self.apple_position[1]+10),(0,0,255),3)
        # Display Snake
        for position in self.snake_position:
            cv2.rectangle(self.img,(position[0],position[1]),(position[0]+10,position[1]+10),(0,255,0),3)
        
        # Takes step after fixed time
        t_end = time.time() + 0.05
        k = -1
        while time.time() < t_end:
            if k == -1:
                k = cv2.waitKey(1)
            else:
                continue

        button_direction = action
        # Change the head position based on the button direction
        if button_direction == 1:
            self.snake_head[0] += 10
        elif button_direction == 0:
            self.snake_head[0] -= 10
        elif button_direction == 2:
            self.snake_head[1] += 10
        elif button_direction == 3:
            self.snake_head[1] -= 10


        apple_reward = 0
        # Increase Snake length on eating apple
        if self.snake_head == self.apple_position:
            self.apple_position, self.score = collision_with_apple(self.apple_position, self.score)
            self.snake_position.insert(0,list(self.snake_head))
            apple_reward = 10000

        else:
            self.snake_position.insert(0,list(self.snake_head))
            self.snake_position.pop()
        
        # On collision kill the snake and print the score
        if collision_with_boundaries(self.snake_head) == 1 or collision_with_self(self.snake_position) == 1:
            font = cv2.FONT_HERSHEY_SIMPLEX
            self.img = np.zeros((500,500,3),dtype='uint8')
            cv2.putText(self.img,'Your Score is {}'.format(self.score),(140,250), font, 1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('a',self.img)
            self.done = True
        


        euclidean_dist_to_apple = np.linalg.norm(np.array(self.snake_head) - np.array(self.apple_position))

        self.total_reward = ((250 - euclidean_dist_to_apple) + apple_reward)/100

        print(self.total_reward)


        self.reward = self.total_reward - self.prev_reward
        self.prev_reward = self.total_reward

        if self.done:
            self.reward = -10
        info = {}


        head_x = self.snake_head[0]
        head_y = self.snake_head[1]

        snake_length = len(self.snake_position)
        apple_delta_x = self.apple_position[0] - head_x
        apple_delta_y = self.apple_position[1] - head_y

        # create observation:

        observation = [head_x, head_y, apple_delta_x, apple_delta_y, snake_length] + list(self.prev_actions)
        observation = np.array(observation)

        return observation, self.total_reward, self.done, info

    def reset(self):
        self.img = np.zeros((500,500,3),dtype='uint8')
        # Initial Snake and Apple position
        self.snake_position = [[250,250],[240,250],[230,250]]
        self.apple_position = [random.randrange(1,50)*10,random.randrange(1,50)*10]
        self.score = 0
        self.prev_button_direction = 1
        self.button_direction = 1
        self.snake_head = [250,250]

        self.prev_reward = 0

        self.done = False

        head_x = self.snake_head[0]
        head_y = self.snake_head[1]

        snake_length = len(self.snake_position)
        apple_delta_x = self.apple_position[0] - head_x
        apple_delta_y = self.apple_position[1] - head_y

        self.prev_actions = deque(maxlen = SNAKE_LEN_GOAL)  # however long we aspire the snake to be
        for i in range(SNAKE_LEN_GOAL):
            self.prev_actions.append(-1) # to create history

        # create observation:
        observation = [head_x, head_y, apple_delta_x, apple_delta_y, snake_length] + list(self.prev_actions)
        observation = np.array(observation)

        return observation

In [5]:
models_dir = f"models/{int(time.time())}/"
logdir = f"logs/{int(time.time())}/"

if not os.path.exists(models_dir):
    os.makedirs(models_dir)

if not os.path.exists(logdir):
    os.makedirs(logdir)

In [6]:
env = SnekEnv()
env.reset()

# We can change to any of the models provided by SB3, unless the model adheres to action and observation space
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=logdir)

TIMESTEPS = 10000

for i in range(1,100000000):
    model.learn(total_timesteps=TIMESTEPS, reset_num_timesteps=False, tb_log_name=f"PPO")
    model.save(f"{models_dir}/{TIMESTEPS*i}")

env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to logs/1654437838/PPO_0
0.7535750803427018
0.6560911085414225
0.55835121610524
0.5764615938328654
0.55835121610524
1.495012437887911
1.5944614861862585
1.693774225170145
1.5944614861862585
0.8
0.8875484503402902
0.8356683022906762
0.7507144315464097
0.6972243622680054
0.7507144315464097
0.5201010126776668
0.44817154713168095
0.37867965644035734
0.30682878005386927
0.23725830020304783
0.16547649401424963
0.23284319024907318
0.29772844544547583
0.23284319024907318
1.7
1.693774225170145
1.7928932188134525
1.8
1.7928932188134525
0.49750156054992145
0.5
-0.2018512172212593
-0.2658633371878659
-0.34253408071037883
-0.2802877548915689
-0.22029410174708858
-0.14007575648881698
-0.08069758011278794
0.0
0.079256312617959
0.15692509722800366
0.09168108424154099
0.16547649401424963
0.23725830020304783
0.30682878005386927
0.23725830020304783
0.3159670332158444
0.3412966855077096
0.4751543268683412

-0.14764045897474545
-0.16833281282526685
-0.14764045897474545
0.12302713519905752
0.4751543268683412
0.55835121610524
0.3159670332158444
0.4119386982178901
0.5075141154828725
0.6026334038989725
0.6318458307730594
0.53531172956115
0.5075141154828725
0.6026334038989725
0.6318458307730594
0.6560911085414225
0.675171240910534
0.5764615938328654
0.675171240910534
0.6560911085414225
0.29772844544547583
0.23284319024907318
0.4384471871911697
0.46039219456288605
1.7189750324093347
1.778889744907202
1.859687576256715
1.7928932188134525
1.7189750324093347
1.6397674732957372
1.7189750324093347
1.867544467966324
1.891723746970178
1.7928932188134525
1.771989011071948
1.867544467966324
1.9614835192865496
1.9900980486407216
2.0
1.9
2.0
0.21745755789733465
0.6615223689149764
0.6972243622680054
0.6026334038989725
0.6318458307730594
0.6026334038989725
0.8721179403900294
0.7737323498367931
1.0577794898144044
1.0
1.0577794898144044
0.5895026825457199
0.490024875775822
-0.24590604354919607
-0.301785145224

1.7384226894136092
1.645599625468247
1.605572809000084
1.693774225170145
1.7384226894136092
1.693774225170145
0.16547649401424963
0.26393202250021036
0.6972243622680054
0.727995485333065
0.7535750803427018
0.7737323498367931
0.7535750803427018
0.9735662477526253
0.026136624629403683
0.0
0.026136624629403683
0.23284319024907318
0.29772844544547583
0.2196491498017241
0.24389716546430434
0.6
0.5973702409559553
0.6972243622680054
0.7970613634073598
0.8968780458118601
0.7970613634073598
-0.049509756796392426
0.016051530325155737
-0.5413812651491099
-0.6112698372208092
-0.6827660925679094
-0.7557641192199412
-0.8241540277189324
-0.8941125496954282
-0.8241540277189324
1.0857864376269049
1.0133931252681494
1.0787329596448105
1.0133931252681494
0.30682878005386927
0.26393202250021036
0.1740593300773986
0.08339080528108553
0.1740593300773986
0.8236945385759791
0.9188611699158102
0.8844505578596489
0.9768453788272182
0.9347524157501473
0.844705464275315
0.7535750803427018
0.6615223689149764
0.621

-0.12488094968133737
-0.2018512172212593
1.778889744907202
1.859687576256715
1.934314575050762
1.859687576256715
1.778889744907202
1.8291796067500632
1.7384226894136092
1.693774225170145
1.605572809000084
1.693774225170145
0.026136624629403683
0.48753882025018924
0.5764615938328654
0.664244024931418
0.7507144315464097
0.6972243622680054
0.7795349465914745
0.8356683022906762
0.7795349465914745
0.030182192954306117
0.08132267551043498
0.12934608177406035
0.08132267551043498
0.46039219456288605
0.4776251583843316
0.46039219456288605
0.3622441673568051
0.4119386982178901
0.38103798995829097
0.2864056378821346
0.3159670332158444
0.3412966855077096
0.3159670332158444
0.09168108424154099
0.16547649401424963
0.23725830020304783
0.30682878005386927
0.23284319024907318
0.15692509722800366
0.2196491498017241
0.1399152557588107
0.079256312617959
0.0
0.079256312617959
2.052786404500042
1.9614835192865496
0.026136624629403683
0.0
-0.029822128134703405
0.0
-0.12488094968133737
-0.029822128134703405
-

-0.049509756796392426
-0.14007575648881698
0.7111456180001682
0.8
0.7507144315464097
0.8356683022906762
0.9188611699158102
0.8356683022906762
0.4384471871911697
0.37397083745307014
0.29772844544547583
0.2196491498017241
0.27963966888254815
0.19782711335573225
0.11462791162468733
0.05868887685325944
-0.02388589282479245
-0.1076809620810593
-0.02388589282479245
1.2350889359326482
1.3295300089280375
1.2916954026405427
1.3295300089280375
1.693774225170145
1.6397674732957372
1.7189750324093347
1.6397674732957372
0.5895026825457199
1.891723746970178
1.7928932188134525
1.693774225170145
1.7928932188134525
1.8291796067500632
1.91690481051547
0.55835121610524
0.6398924762261725
0.6132037735886792
0.527691707668398
0.5764615938328654
0.664244024931418
0.7111456180001682
0.664244024931418
0.1399152557588107
-0.547950130825634
-0.48328677803525977
-0.547950130825634
2.139444872453601
2.0757359312880714
2.139444872453601
0.7535750803427018
1.495012437887911
1.5944614861862585
1.693774225170145
1.77

...

2.4
100.43844718719117
0.5
0.55835121610524
0.6132037735886792
0.664244024931418
0.7111456180001682
0.7535750803427018
0.7911992509364939
0.8236945385759791
0.8507577497529357
0.8721179403900294
0.8875484503402902
0.8968780458118601
0.9
0.8968780458118601
0.8875484503402902
0.8721179403900294
0.8507577497529357
0.8236945385759791
0.7911992509364939
0.7535750803427018
0.7111456180001682
0.664244024931418
0.6132037735886792
0.527691707668398
0.4751543268683412
0.39049768902710125
0.3366692347216065
0.4191347953315187
0.4751543268683412
0.4191347953315187
2.217157287525381
2.139444872453601
2.183772233983162
2.2
2.1
2.087689437438234
1.9900980486407216
1.891723746970178
1.867544467966324
1.771989011071948
1.7928932188134525
1.693774225170145
1.5944614861862585
1.495012437887911
1.5
1.495012437887911
1.480196097281443
1.455969349108945
1.4229670385730993
1.3295300089280375
1.2916954026405427
1.2470035913858333
1.1961595189594703
1.1107556010550195
1.0235176939766601
0.9735662477526253
0.88

1.33380962103094
1.4183346173608031
1.5
1.5780455542707115
1.6514718625761429
1.7189750324093347
1.7928932188134525
1.859687576256715
1.934314575050762
2.0
2.052786404500042
2.139444872453601
2.217157287525381
2.2763932022500213
2.3
2.4
98.5439919110295
-1.3600518131237567
-1.2643060449437422
-1.1687872655688283
-1.0735136770411275
-0.9785054261852173
-0.8837848631377261
-0.7893768406797057
-0.6953090617340917
-0.6016124838541645
-0.5083217912982644
-0.41547594742265004
-0.32311884269862107
-0.2313000567495328
-0.14007575648881698
-0.049509756796392426
0.040325224750231425
0.12934608177406035
0.21745755789733465
0.304550159989985
0.39049768902710125
0.4751543268683412
0.55835121610524
0.6398924762261725
0.7195506185235143
0.7970613634073598
0.8721179403900294
0.9443650813895954
1.0133931252681494
1.0787329596448105
1.1398529491264557
1.1961595189594703
1.2470035913858333
1.2916954026405427
1.3295300089280375
1.359824574900862
1.381966011250105
1.395463898281274
1.4
1.395463898281274
1.

0.55835121610524
0.46039219456288605
0.55835121610524
0.4384471871911697
0.530228439640779
0.6211705771944065
0.7111456180001682
0.8
0.8875484503402902
0.9735662477526253
1.0577794898144044
1.1398529491264557
1.2193751525134302
1.2958405421207704
1.368629150101524
1.436985418726535
1.5
1.5566018867943396
1.605572809000084
1.645599625468247
1.675378874876468
1.693774225170145
1.7
1.693774225170145
1.5944614861862585
1.5780455542707115
1.675378874876468
1.645599625468247
1.605572809000084
1.5566018867943396
1.4704369859013
1.5151142198203893
1.5513167019494865
1.455969349108945
1.480196097281443
1.495012437887911
1.5
1.495012437887911
1.480196097281443
1.455969349108945
1.4229670385730993
1.381966011250105
1.33380962103094
1.2793444384266297
1.2193751525134302
1.1398529491264557
1.0787329596448105
1.1546375952926289
1.0857864376269049
1.0133931252681494
0.9443650813895954
0.8721179403900294
0.8029437251522858
0.8721179403900294
0.7970613634073598
0.8598780533143275
0.7970613634073598
1.0

1.1546375952926289
1.0857864376269049
1.0133931252681494
0.9443650813895954
0.8721179403900294
0.7970613634073598
0.7195506185235143
0.6398924762261725
0.55835121610524
0.5
0.4384471871911697
0.3599065440967303
0.29772844544547583
0.23284319024907318
0.16547649401424963
0.09168108424154099
0.15692509722800366
0.079256312617959
0.0
-0.06124969497313941
-0.12488094968133737
-0.19072480941474226
-0.2586228448267445
-0.331960451701259
-0.4
-0.46984848098349974
-0.5413812651491099
-0.6144823004794876
2.183772233983162
2.2
2.1
2.0
1.9
1.8
1.7
1.693774225170145
1.675378874876468
1.5780455542707115
1.5513167019494865
1.455969349108945
1.4229670385730993
1.381966011250105
1.33380962103094
1.2793444384266297
1.33380962103094
1.0682178936723647
1.0857864376269049
1.09643311523818
1.1
1.09643311523818
1.0857864376269049
0.9867254049578443


KeyboardInterrupt: 

In [8]:
env.close()